In [79]:
"""
過去のセリーグ、パリーグのMVPの選手の打撃成績から、2019年のMVPが本当に坂本勇人選手なのか、検証していきます。
過去のMVP選手とそうでない選手の打撃成績をLogisticRegressionの機械学習モデルに学習させ、2019年のプロ野球の打者の成績をテストデータとして、2019年のMVP選手(とそうでない選手たち)を予測させます。
"""

'\n過去のセリーグ、パリーグのMVPの選手の打撃成績から、2019年のMVPが本当に坂本勇人選手なのか、検証していきます。\n過去のMVP選手とそうでない選手の打撃成績をLogisticRegressionの機械学習モデルに学習させ、2019年のプロ野球の打者の成績をテストデータとして、2019年のMVP選手(とそうでない選手たち)を予測させます。\n'

In [80]:
import numpy as np
import pandas as pd 

from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler


In [81]:
# CSVデータファイルをpandas.DataFrameの形式に
dfs = [pd.read_csv('data/batter_' + str(yr) + '.csv') for yr in range(2009, 2020)]

In [82]:
type(dfs)

list

In [83]:
type(dfs[0])

pandas.core.frame.DataFrame

In [84]:
np.isnan(dfs[0]['試合'])

0     False
1     False
2     False
3     False
4     False
      ...  
56    False
57    False
58    False
59    False
60    False
Name: 試合, Length: 61, dtype: bool

In [85]:
#セ・リーグ、パ・リーグの両MVPがどちらとも投手だった、2014年の行は削除し、分析したい2019年のデータも入れない。

In [86]:
"""
作成した pandas.DataFrame を用いて、訓練データを作成します。
2009年から、2018年までのデータを結合させます。この際に、セ・リーグ、パ・リーグの両MVPがどちらとも投手受賞だった、2014年の行は削除し、テストデータとして用いる2019年のデータも外します。
"""
whole_year_array = np.concatenate((dfs[0], dfs[1], dfs[2], dfs[3], dfs[4], dfs[6], dfs[7], dfs[8], dfs[9]), axis = 0)

In [87]:
whole_year_array

array([['田中 賢介', '日本ハム', 144, ..., 17, 0, 0],
       ['井端 弘和', '中日', 144, ..., 8, 13, 0],
       ['中島 裕之', '西武', 144, ..., 0, 17, 0],
       ...,
       ['中島 卓也', '日本ハム', 132, ..., 22, 4, 0],
       ['野間 峻祥', '広島', 126, ..., 4, 2, 0],
       ['松山 竜平', '広島', 124, ..., 0, 10, 0]], dtype=object)

In [88]:
type(whole_year_array)

numpy.ndarray

In [89]:
df_whole_year = pd.DataFrame(data = whole_year_array, columns = dfs[0].columns)

In [90]:
df_whole_year

,選手名,チーム,試合,打席数,打数,安打,本塁打,打点,盗塁,四球,死球,三振,犠打,併殺打,MVP
0,田中 賢介,日本ハム,144,680,575,163,3,49,31,79,5,105,17,0,0
1,井端 弘和,中日,144,657,569,174,5,39,13,72,6,66,8,13,0
2,中島 裕之,西武,144,648,560,173,22,92,20,75,10,113,0,17,0
3,片岡 易之,西武,139,646,588,153,13,58,51,36,7,72,12,8,0
4,栗山 巧,西武,140,643,569,152,12,57,18,53,10,106,8,9,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,梅野 隆太郎,阪神,132,455,386,100,8,47,5,39,1,67,28,6,0
502,島内 宏明,楽天,103,453,394,115,11,53,11,47,6,45,3,10,0
503,中島 卓也,日本ハム,132,449,391,102,1,23,29,35,0,88,22,4,0
504,野間 峻祥,広島,126,447,405,116,5,46,17,30,6,69,4,2,0


In [91]:
df_whole_year.columns

Index(['選手名', 'チーム', '試合', '打席数', '打数', '安打', '本塁打', '打点', '盗塁', '四球', '死球',
       '三振', '犠打', '併殺打', 'MVP'],
      dtype='object')

In [92]:
"""
先ほど作成したpandasのDataFrameから、選手名とチーム、MVPの列を外し（indexで選手の識別可能）それを訓練データ X_train とし、 'MVP'の列を正解ラベルとして Y_trainに格納した。

テストデータは2019年のデータであるため、訓練データと同様に、dfs[10]の選手名とチーム、MVPの列を外したデータをテストデータ X_test 、 MVPの列を正解ラベルとして、Y_test に格納した。
"""

X_train = df_whole_year[['試合', '打席数', '打数', '安打', '本塁打', '打点', '盗塁', '四球', '死球',
       '三振', '犠打', '併殺打']]

Y_train = df_whole_year[['MVP']]
Y_train = Y_train.astype('int')

X_test = dfs[10][['試合', '打席数', '打数', '安打', '本塁打', '打点', '盗塁', '四球', '死球',
       '三振', '犠打', '併殺打']]

Y_test = dfs[10][['MVP']]
Y_test = Y_test.astype('int')

In [93]:
# ロジスティック回帰クラスの初期化(インスタンス生成)と学習
model = LogisticRegression()
model.fit(X_train, Y_train)

LogisticRegression()

In [94]:
# 予測の結果,mvpは鈴木誠也選手であることがわかりました。
model.predict(X_test)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [95]:
# 予測の正解率
print('正解率(train):{:.3f}'.format(model.score(X_train, Y_train)))

正解率(train):0.980


In [96]:
# 予測の正解率
print('正解率(test):{:.3f}'.format(model.score(X_test, Y_test)))

正解率(test):0.950


In [97]:
model.predict(X_train)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,

In [98]:
# スケーリングによる予測精度の向上を期待する
# 標準化処理 
sc = StandardScaler()
sc.fit(X_train)
X_train_std = sc.transform(X_train)
X_test_std = sc.transform(X_test)

model = LogisticRegression()
model.fit(X_train_std, Y_train)


LogisticRegression()

In [99]:
model.predict(X_test_std)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [100]:
model.score(X_train_std, Y_train)

0.9782608695652174

In [101]:
# スケーリングによって、正答率は変わらず、MVPも鈴木誠也選手で変わりませんでした。
model.score(X_test_std, Y_test)

0.95

In [102]:
dfs[10][14:15]

,選手名,チーム,試合,打席数,打数,安打,本塁打,打点,盗塁,四球,死球,三振,犠打,併殺打,MVP
14,鈴木 誠也,広島,140,612,499,167,28,87,25,103,7,81,0,3,0
